<a href="https://colab.research.google.com/github/Rafaelfyh/Challenge_DataScience/blob/main/Din%C3%A2mica_Spark_Grupo_21_DEX4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DEX 4 Grupo 21, nomes e ID: 27710 Andreia Sampaio da costa, 13720 Vinicius Azevedo dos santos, 15084 Rafael Queiroz de Oliveira

# **SETUP**

## Spark UI

In [1]:
!pip install -q pyspark
!pip install -q pyngrok

     |████████████████████████████████| 281.4 MB 37 kB/s 
     |████████████████████████████████| 199 kB 36.2 MB/s 
     |████████████████████████████████| 745 kB 4.9 MB/s 


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.config('spark.ui.port', '4050').getOrCreate()
spark

In [3]:
# Fazer login no site https://dashboard.ngrok.com/get-started/setup para obter autenticação própria
ngrok_token = '27WqUGZ1SkzPl4bQunMXJyasbWZ_5pCgLCsppL7ufk9rzK7j4'

In [4]:
get_ipython().system_raw(f'ngrok authtoken {ngrok_token}')
get_ipython().system_raw('ngrok http 4050 &')
!sleep 3
print('URL para interface Spark:')
!curl -s http://localhost:4040/api/tunnels | grep -Po 'public_url":"(?=https)\K[^"]*'

URL para interface Spark:


## Libraries

In [86]:
import pandas as pd
from google.colab import files

import pyspark.sql.functions as F
from pyspark.sql.window import Window

# Load data

In [6]:
# Load file
_ = files.upload()

Saving vgsales.csv to vgsales.csv


## Data Description

This dataset contains records of popular video games in North America, Japan, Europe and other parts of the world. Every video game in this dataset has at least 100k global sales.

[Source](https://www.kaggle.com/datasets/gregorut/videogamesales/code?datasetId=284&sortBy=voteCount) of dataset.

## Data Dictionary

| Column       | Explanation                                               |
|:-------------|:----------------------------------------------------------|
| Rank         | Ranking of overall sales                                  |
| Name         | Name of the game                                          |
| Platform     | Platform of the games release (i.e. PC,PS4, etc.)         |
| Year         | Year the game was released in                             |
| Genre        | Genre of the game                                         |
| Publisher    | Publisher of the game                                     |
| NA_Sales     | Number of sales in North America (in millions)            |
| EU_Sales     | Number of sales in Europe (in millions)                   |
| JP_Sales     | Number of sales in Japan (in millions)                    |
| Other_Sales  | Number of sales in other parts of the world (in millions) |
| Global_Sales | Number of total sales (in millions)                       |

# Questões

## Questão 1

Leia os dados com Spark, certificando-se que

1. A tabela tenha duas partições e

2. As colunas da tabela tenham este respectivo schema:

|Column|Data type|
|---|---|
|Rank|integer|
|Name|string|
|Platform|string|
|Year|integer|
|Genre|string|
|Publisher|string|
|NA_Sales|double|
|EU_Sales|double|
|JP_Sales|double|
|Other_Sales|double|
|Global_Sales|double|

In [8]:
df = spark.read.csv('vgsales.csv', header=True, inferSchema=True)

In [18]:
df = df.withColumn('Year', F.col('Year').cast('int'))

In [19]:
df.rdd.getNumPartitions()

2

In [20]:
df = df.repartition(2)
df.rdd.getNumPartitions()

2

In [21]:
df.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Platform: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Publisher: string (nullable = true)
 |-- NA_Sales: double (nullable = true)
 |-- EU_Sales: double (nullable = true)
 |-- JP_Sales: double (nullable = true)
 |-- Other_Sales: double (nullable = true)
 |-- Global_Sales: double (nullable = true)



In [22]:
df.pandas_api().head() 

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,2343,NFL 2K2,PS2,2001,Sports,Sega,0.44,0.34,0.0,0.11,0.89
1,3886,Prince of Persia Trilogy,PS3,2010,Action,Ubisoft,0.17,0.25,0.0,0.09,0.51
2,8880,Hot Wheels: Beat That!,Wii,2007,Racing,Activision,0.13,0.00,0.0,0.01,0.15
3,1190,Twisted Metal,PS,1995,Action,Sony Computer Entertainment,1.32,0.19,0.0,0.05,1.56
4,9936,Turok: Evolution,GBA,2002,Shooter,Acclaim Entertainment,0.08,0.03,0.0,0.00,0.12


## Questão 2

Construa uma tabela com:
- contagem das observações
- média
- desvio padrão
- valor mínimo
- valor máximo

para todas as coluna de vendas (que possuem "Sales" no nome).

**Dica:** uma **única função** consegue calcular todos estes valores.

In [29]:
lista_sales = [coluna for coluna in df.columns if 'Sales' in coluna]
df.select(*lista_sales).describe().show()

+-------+------------------+-------------------+-------------------+--------------------+------------------+
|summary|          NA_Sales|           EU_Sales|           JP_Sales|         Other_Sales|      Global_Sales|
+-------+------------------+-------------------+-------------------+--------------------+------------------+
|  count|             16598|              16598|              16598|               16598|             16598|
|   mean|0.2646674298108155|0.14665200626581024|0.07778166044101428|0.048063019640918934|0.5374406555006714|
| stddev|0.8166830292988791| 0.5053512312869121| 0.3092906480822032| 0.18858840291271461|1.5550279355699124|
|    min|               0.0|                0.0|                0.0|                 0.0|              0.01|
|    max|             41.49|              29.02|              10.22|               10.57|             82.74|
+-------+------------------+-------------------+-------------------+--------------------+------------------+



## Questão 3

Para cada variável categórica do tipo string, calcule quantas categorias distintas estão presentes.

In [30]:
lista_dados = [field['name'] for field in df.schema.jsonValue()['fields'] if field['type'] == 'string']

df.select(*[F.count_distinct(col) for col in lista_dados]).show()

+--------------------+------------------------+---------------------+-------------------------+
|count(DISTINCT Name)|count(DISTINCT Platform)|count(DISTINCT Genre)|count(DISTINCT Publisher)|
+--------------------+------------------------+---------------------+-------------------------+
|               11493|                      31|                   12|                      579|
+--------------------+------------------------+---------------------+-------------------------+



## Questão 4

Qual plataforma vendeu mais, mundialmente, considerando todo o período histórico diponível dos dados?  
Mostre as top 10 plataformas em vendas.

In [36]:
(
  df
 .groupby('Platform')
 .agg(
    F.sum('Global_Sales').alias('Global_Sales_sum')
 )
 .orderBy(F.desc('Global_Sales_sum'))
.limit(10)
 .show()
)

+--------+------------------+
|Platform|  Global_Sales_sum|
+--------+------------------+
|     PS2|1255.6399999999956|
|    X360| 979.9600000000003|
|     PS3| 957.8400000000025|
|     Wii| 926.7099999999973|
|      DS| 822.4899999999982|
|      PS| 730.6599999999992|
|     GBA|318.49999999999943|
|     PSP| 296.2799999999996|
|     PS4| 278.0999999999999|
|      PC|258.82000000000085|
+--------+------------------+



## Questão 5

Faça uma tabela com os jogos que aparecem em múltiplas plataformas, ordene de forma que os jogos com mais plataformas apareçam primeiro e responda:
- Qual jogo aparece em mais plataformas? Em quantas plataformas?

In [48]:
(
  df
 .groupby('Name')
 .agg(
    F.countDistinct('Platform').alias('n_platform')
 )
 .where('n_platform>1')
 .orderBy(F.desc('n_platform'))
 .limit(1)
 .show()
)

+--------------------+----------+
|                Name|n_platform|
+--------------------+----------+
|Need for Speed: M...|        10|
+--------------------+----------+



**Resposta:** Need for Speed: Most Wanted. Aparece em 10 plataformas.

## Questão 6

Utilize a API do Pandas no Spark para calcular a soma das vendas globais para cada ano e gênero de jogo. Faça então um gráfico de linhas com os anos no eixo `x`, as vendas no eixo `y`, de forma que cada linha corresponda a um gênero de jogo.

**Dica:** após o cálculo, passar os dados para Pandas antes da plotagem, ou plotar diretamente aproveitando os métodos da classe `pyspark.pandas.DataFrame`.

Analise o gráfico e responda:
- Entre 1980 e 1990, quais gêneros mais venderam?
- Entre 2000 e 2015, quais gêneros mais venderam?

In [66]:
(
  df
  .pandas_api()
 .groupby(['Genre', 'Year'])[['Global_Sales']].sum() 
 .sort_index()
 .reset_index()
 .plot.line(x='Year', y='Global_Sales', color='Genre')
)


**Resposta:**
- Entre 1980 e 1990 sobressaíram as vendas de plataforma, tiro (shooter) e puzzle.
- Entre 2000 e 2015 teve vendas elevadas de jogos de ação e esporte.

## Questão 7

Registre a tabela usando `createOrReplaceTempView` e faça uso da linguagem SQL criar uma tabela que:
- Considere apenas os anos da década de 90 e 
- Agrupe por ano para responder quantos % cada região teve do total de vendas (vendas globais).

Salve o resultado desta query em uma variável chamada `df_questao7`.

Após isso, execute o seguinte comando `df_questao7.pandas_api().set_index('YEAR').style.background_gradient(cmap='Oranges')` e responda:
- Qual/quais regiões tiveram, relativamente (comparado às vendas globais), mais vendas no fim da década de noventa do que no início?








In [89]:
df.createOrReplaceTempView('anos_noventa')

In [92]:
Query = """
SELECT
  YEAR,
  ROUND(SUM(NA_Sales)/SUM(Global_Sales)*100) as sum_na,
  ROUND(SUM(EU_Sales)/SUM(Global_Sales)*100) as sum_eu,
  ROUND(SUM(JP_Sales)/SUM(Global_Sales)*100) as sum_jp,
  ROUND(SUM(Other_Sales)/SUM(Global_Sales)*100) as sum_ot
FROM
  anos_noventa
WHERE YEAR BETWEEN 1990 and 2000
GROUP BY
  YEAR
ORDER BY
  YEAR
"""

df_questao7=spark.sql(Query)
df_questao7.show()

+----+------+------+------+------+
|YEAR|sum_na|sum_eu|sum_jp|sum_ot|
+----+------+------+------+------+
|1990|  52.0|  15.0|  30.0|   3.0|
|1991|  40.0|  12.0|  46.0|   2.0|
|1992|  44.0|  15.0|  38.0|   2.0|
|1993|  33.0|  10.0|  55.0|   2.0|
|1994|  36.0|  19.0|  43.0|   3.0|
|1995|  28.0|  17.0|  52.0|   3.0|
|1996|  44.0|  24.0|  29.0|   4.0|
|1997|  47.0|  24.0|  24.0|   5.0|
|1998|  50.0|  26.0|  20.0|   4.0|
|1999|  50.0|  25.0|  21.0|   4.0|
|2000|  47.0|  26.0|  21.0|   6.0|
+----+------+------+------+------+



In [93]:
df_questao7.pandas_api().set_index('YEAR').style.background_gradient(cmap='Oranges')

,sum_na,sum_eu,sum_jp,sum_ot
YEAR,,,,
1990,52.000000,15.000000,30.000000,3.000000
1991,40.000000,12.000000,46.000000,2.000000
1992,44.000000,15.000000,38.000000,2.000000
1993,33.000000,10.000000,55.000000,2.000000
1994,36.000000,19.000000,43.000000,3.000000
1995,28.000000,17.000000,52.000000,3.000000
1996,44.000000,24.000000,29.000000,4.000000
1997,47.000000,24.000000,24.000000,5.000000
1998,50.000000,26.000000,20.000000,4.000000


In [94]:
df_questao7.printSchema()

root
 |-- YEAR: integer (nullable = true)
 |-- sum_na: double (nullable = true)
 |-- sum_eu: double (nullable = true)
 |-- sum_jp: double (nullable = true)
 |-- sum_ot: double (nullable = true)



## Questão 8

Calcule quantas vezes cada região teve vendas superiores às demais.

In [90]:
df.pandas_api().head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,2343,NFL 2K2,PS2,2001,Sports,Sega,0.44,0.34,0.0,0.11,0.89
1,3886,Prince of Persia Trilogy,PS3,2010,Action,Ubisoft,0.17,0.25,0.0,0.09,0.51
2,8880,Hot Wheels: Beat That!,Wii,2007,Racing,Activision,0.13,0.00,0.0,0.01,0.15
3,1190,Twisted Metal,PS,1995,Action,Sony Computer Entertainment,1.32,0.19,0.0,0.05,1.56
4,9936,Turok: Evolution,GBA,2002,Shooter,Acclaim Entertainment,0.08,0.03,0.0,0.00,0.12


In [99]:
lista_regiao=['NA_Sales',	'EU_Sales',	'JP_Sales',	'Other_Sales']


(
df
.withColumn(
'max_value',F.greatest(*lista_regiao))
.withColumn('maior_regiao',
  F.when(F.col('NA_Sales')==F.col('max_value'), 'NA_Sales')
  .when(F.col('EU_Sales')==F.col('max_value'), 'EU_Sales')
  .when(F.col('JP_Sales')==F.col('max_value'), 'JP_Sales')
  .otherwise('Other_Sales')
)
.groupby('maior_regiao')
.count()
.show()
)

+------------+-----+
|maior_regiao|count|
+------------+-----+
|    EU_Sales| 2380|
|    JP_Sales| 4029|
|    NA_Sales|10113|
| Other_Sales|   76|
+------------+-----+



## Questão 9

Construe uma tabela que mostre a diferença do total de vendas em um década com a década anterior e responda:
- Qual década apresentou a **menor diferença** comparada à década anterior?

## Questão 10 - PLUS

Utilizando apenas a sintaxe do pyspark, faça uma função que calcule a moda do dataframe para um grupo de colunas que o usuário vai especificar. Além da moda, o dataframe de saída deve conter a quantidade de vezes que a moda aparece, e tambem se é multimodal.

Sua função deve receber como entrada:
- O pyspark dataframe
- A coluna-alvo que desejamos saber a moda e
- Os grupos (colunas do dataframe) que iremos considerar para calcular a moda 

O resultado será uma linha por grupo, contendo a identificação do grupo, a moda, a quantidade de vezes que apareceu, e se é um caso multimodal.

Por exemplo, considerando o seguinte dataframe:

```
df_test = spark.createDataFrame([
  (2.4,'A','A1'), (2.4,'A','A1'), (2.5,'A','A1'), (2.6,'A','A1'), (2.7,'A','A1'),
  (2.4,'B','A1'), (2.5,'B','A1'), (2.5,'B','A1'), (2.6,'B','A1'), (2.4,'B','A1')
], ('values','inner_group', 'main_group')).select('main_group', 'inner_group', 'values')

df_test.show()
```

```
+----------+-----------+------+
|main_group|inner_group|values|
+----------+-----------+------+
|        A1|          A|   2.4|
|        A1|          A|   2.4|
|        A1|          A|   2.5|
|        A1|          A|   2.6|
|        A1|          A|   2.7|
|        A1|          B|   2.4|
|        A1|          B|   2.5|
|        A1|          B|   2.5|
|        A1|          B|   2.6|
|        A1|          B|   2.4|
+----------+-----------+------+
```

Ao aplicar a função:  
`calculate_mode(df=df_test, target_col='values', group_cols=['main_group','inner_group']).show()`

o retorno deve ser:

```
+----------+-----------+----+-----------+----------+
|main_group|inner_group|mode|mode_counts|multimodal|
+----------+-----------+----+-----------+----------+
|        A1|          A| 2.4|          2|     false|
|        A1|          B| 2.4|          2|      true|
+----------+-----------+----+-----------+----------+
```

pois no grupo `A1-A` o valor 2.4 é o que mais aparece (2x), e nenhum outro valor aparece duas vezes também. Já no grupo `A1-B` o valor 2.4 também é o que mais aparece (2X), porém em conjunto com o 2.5, que também aparece duas 2X. Neste caso, devemos reportar apenas uma das modas e informar que é multimodal.